In [1]:
import os

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Info'].tolist()

总数量：10


['https://www.rockauto.com/en/moreinfo.php?pk=8304960&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=11158045&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=8304968&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=8304972&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=8304976&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=9658980&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=9658984&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=8304980&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=10110000&cc=0&pt=5756',
 'https://www.rockauto.com/en/moreinfo.php?pk=8304984&cc=0&pt=5756']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

b = 0
while True:
    b += 1
    print(b)
    try:
        request_url = input_.loc[a, 'Info']
        
        resp = requests.get(request_url,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(10, 10))
        
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            if html.xpath('//span[@class="la-btn-v"]/text()')[0] == 'Continue Shopping':
                with open('./info.html', 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except KeyboardInterrupt:
        break
    except:
        continue

resp

1
2
3
4
5


<Response [200]>

In [8]:
import re

list_tr, dict_count = html.xpath('//table[@class="moreinfotable"]/tr')[1:] if html.xpath('//table[@class="moreinfotable"]/tr') else [], {}
for i, tr in enumerate(list_tr):
    list_td = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
    list_td = [re.sub(r' +', ' ', td).strip() for td in list_td]
    if list_td[0] in dict_count:
        dict_count[list_td[0]] += 1
    else:
        dict_count[list_td[0]] = 1

dict_specification, dict_no = {}, {}
for i, tr in enumerate(list_tr):
    dict_specification[str(i)] = {}

    list_td = [re.sub(r'<.*?>', '', etree.tostring(td).decode('utf-8')) for td in tr.xpath('./td')]
    list_td = [re.sub(r' +', ' ', td).strip() for td in list_td]
    for j, td in enumerate(list_td):
        if j == 0:
            if dict_count[td] == 1:
                dict_specification[str(i)][str(j)] = td
            else:
                if td in dict_no:
                    dict_no[td] += 1
                else:
                    dict_no[td] = 1
                dict_specification[str(i)][str(j)] = f'''{td}-{dict_no[td]}'''
        else:
            dict_specification[str(i)][str(j)] = td

    if len(list_td) == 1:
        dict_specification[str(i)]['1'] = True
    elif len(list_td) > 2:
        print('fuck')

dict_specification

{'0': {'0': 'Camshaft Sprocket Material', '1': 'High Alloy Steel'},
 '1': {'0': 'Keyway Adjustable', '1': 'No'},
 '2': {'0': 'Sprockets Included', '1': 'Yes'},
 '3': {'0': 'Timing Chain Type', '1': 'Silent'}}

In [9]:
import json

dict_kit, list_tr = {}, html.xpath('//section[contains(@aria-label, "Components") and contains(@aria-label, "kit")]/table/tbody/tr/td/table/tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-b-0") or (contains(@class, "altrow-b-1")))]/tr')
for i, tr in enumerate(list_tr):
    dict_kit[str(i)] = {}

    list_td = tr.xpath('./td')
    dict_kit[str(i)]['Kit_Quantity'] = int(list_td[0].xpath('./text()')[0].strip())

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['Kit_Type'] = list_td[1].xpath('./text()')[0].strip()

    # = = = = = = = = = = = = = = =

    dict_ = json.loads(list_td[2].xpath('./@value')[0])
    dict_kit[str(i)]['Kit_Type Code'] = dict_['parttype']
    dict_kit[str(i)]['Kit_Part Code'] = dict_['partkey']

    # = = = = = = = = = = = = = = =

    dict_2 = json.loads(tr.xpath('./input[contains(@id, "listing_data_supplemental[")]/@value')[0])
    dict_kit[str(i)]['Kit_Part Number'] = dict_2['partnumber']
    dict_kit[str(i)]['Kit_Manufacturer'] = dict_2['catalogname']

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['Kit_Url'] = f'''https://www.rockauto.com/en/parts/{dict_kit[str(i)]['Kit_Manufacturer']},{dict_kit[str(i)]['Kit_Part Number']},{dict_kit[str(i)]['Kit_Type']},{dict_kit[str(i)]['Kit_Type Code']}'''

    # = = = = = = = = = = = = = = =

    list_info = tr.xpath('./div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]')
    dict_kit[str(i)]['Kit_Info'] = list_info[0].xpath('./@href')[0].strip() if list_info else ''
    dict_kit[str(i)]['Kit_Info'] = dict_kit[str(i)]['Kit_Info'] if dict_kit[str(i)]['Kit_Info'].startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={dict_kit[str(i)]['Kit_Part Code']}&cc=0&pt={dict_kit[str(i)]['Kit_Type Code']}&Lennon=1'''

    # = = = = = = = = = = = = = = =
    
    list_src = list_td[3].xpath('./div/input[contains(@id, "jsninlineimg[")]')
    dict_ = json.loads(list_src[0].xpath('./@value')[0]) if list_src else {}

    dict_src = {}
    if dict_:
        list_src = dict_['Slots']
        for j, src in enumerate(list_src):
            dict_src[str(j)] = f'''https://www.rockauto.com{src['ImageData']['Full'].strip()}'''

    dict_kit[str(i)]['Kit_Json_Src'] = json.dumps(dict_src)

    # = = = = = = = = = = = = = = =

    dict_kit[str(i)]['Kit_JOIN_MPNTCPC'] = f'''{dict_kit[str(i)]['Kit_Manufacturer']};{dict_kit[str(i)]['Kit_Part Number']};{dict_kit[str(i)]['Kit_Type Code']};{dict_kit[str(i)]['Kit_Part Code']}'''

dict_kit

{'0': {'Kit_Quantity': 1,
  'Kit_Type': 'Timing Chain',
  'Kit_Type Code': '5724',
  'Kit_Part Code': '8306448',
  'Kit_Part Number': '715F',
  'Kit_Manufacturer': 'MELLING',
  'Kit_Url': 'https://www.rockauto.com/en/parts/MELLING,715F,Timing Chain,5724',
  'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8306448&cc=0&pt=5724',
  'Kit_Json_Src': '{"0": "https://www.rockauto.com/info/583/715F-1-ANG.jpg"}',
  'Kit_JOIN_MPNTCPC': 'MELLING;715F;5724;8306448'},
 '1': {'Kit_Quantity': 1,
  'Kit_Type': 'Timing Chain Guide / Damper',
  'Kit_Type Code': '5728',
  'Kit_Part Code': '8307424',
  'Kit_Part Number': 'BG5515',
  'Kit_Manufacturer': 'MELLING',
  'Kit_Url': 'https://www.rockauto.com/en/parts/MELLING,BG5515,Timing Chain Guide / Damper,5728',
  'Kit_Info': 'https://www.rockauto.com/en/moreinfo.php?pk=8307424&cc=0&pt=5728',
  'Kit_Json_Src': '{"0": "https://www.rockauto.com/info/583/BG5515-1-SID.jpg"}',
  'Kit_JOIN_MPNTCPC': 'MELLING;BG5515;5728;8307424'},
 '2': {'Kit_Quantity': 1

In [10]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Type': input_.loc[a, 'Type'],
                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                         'Part Number': input_.loc[a, 'Part Number'],
                         'Vehicle': input_.loc[a, 'Vehicle'],
                         'OEM': input_.loc[a, 'OEM'],
                         'Picture': input_.loc[a, 'Picture'],
                         'Url': input_.loc[a, 'Url'],
                         'Json_Src': input_.loc[a, 'Json_Src'],
                         'Info': input_.loc[a, 'Info'],
                         'Choose': input_.loc[a, 'Choose'],
                         'Json_Price': input_.loc[a, 'Json_Price'],
                         'Json_Note 1': input_.loc[a, 'Json_Note 1'],
                         'Json_Note 2': input_.loc[a, 'Json_Note 2'],
                         'Json_Specification': json.dumps(dict_specification),
                         'Json_Kit': json.dumps(dict_kit),
                         'Group': input_.loc[a, 'Group'],
                         'Type Code': input_.loc[a, 'Type Code'],
                         'Part Code': input_.loc[a, 'Part Code'],
                         'JOIN_MPNTCPC': input_.loc[a, 'JOIN_MPNTCPC']}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Type,Manufacturer,Part Number,Vehicle,OEM,Picture,Url,Json_Src,Info,Choose,Json_Price,Json_Note 1,Json_Note 2,Json_Specification,Json_Kit,Group,Type Code,Part Code,JOIN_MPNTCPC
0,1,Timing Chain & Component Kit,MELLING,30250S,,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,,"{""0"": ""$171.79""}",{},"{""0"": ""Category: Timing Chain &amp; Component ...","{""0"": {""0"": ""Camshaft Sprocket Material"", ""1"":...","{""0"": {""Kit_Quantity"": 1, ""Kit_Type"": ""Timing ...",Engine,5756,8304960,MELLING;30250S;5756;8304960


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Request_Url'] = request_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_info.xlsx', index=False)

output_correct

,No,Type,Manufacturer,Part Number,Vehicle,OEM,Picture,Url,Json_Src,Info,Choose,Json_Price,Json_Note 1,Json_Note 2,Json_Specification,Json_Kit,Group,Type Code,Part Code,JOIN_MPNTCPC
0,1,Timing Chain & Component Kit,MELLING,30250S,,,,"https://www.rockauto.com/en/parts/melling,3025...","{""0"": ""https://www.rockauto.com/info/583/3-025...",https://www.rockauto.com/en/moreinfo.php?pk=83...,,"{""0"": ""$171.79""}",{},"{""0"": ""Category: Timing Chain &amp; Component ...","{""0"": {""0"": ""Camshaft Sprocket Material"", ""1"":...","{""0"": {""Kit_Quantity"": 1, ""Kit_Type"": ""Timing ...",Engine,5756,8304960,MELLING;30250S;5756;8304960


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_info_error.xlsx', index=False)

output_error

""
